# Tutorial: F3Pipeline con Cache

Este tutorial te guiará a través de un ejemplo sencillo de uso de F3Pipeline, incluyendo la funcionalidad de Cache.

## 1. Importaciones necesarias

In [1]:
from framework3 import F3Pipeline
from framework3.plugins.filters import StandardScalerPlugin
from framework3.plugins.filters import ClassifierSVMPlugin, KnnFilter
from framework3.plugins.metrics import F1
from framework3.base import XYData
from framework3.plugins.filters import Cached
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from rich import print

/home/manuel.couto.pintos/.cache/pypoetry/virtualenvs/framework3-fUhXhdhp-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Preparación de los datos

In [2]:
# Cargar el conjunto de datos Iris
iris = load_iris()
X, y = iris.data, iris.target

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Crear objetos XYData
x_train = XYData("Iris dataset X train", "dataset", X_train)
y_train = XYData("Iris dataset Y train", "dataset", y_train)
x_test = XYData("Iris dataset X test", "dataset", X_test)
y_test = XYData("Iris dataset Y test", "dataset", y_test)

## 3. Creación del pipeline con Cache

In [3]:
# Crear el pipeline con Cache
pipeline = F3Pipeline(
    filters=[
        Cached(
            filter=StandardScalerPlugin(),
            cache_data=True,
            cache_filter=True,
            overwrite=False,
        ),
        ClassifierSVMPlugin(),
    ],
    metrics=[F1()],
)
print(pipeline)

F3Pipeline(
    filters=[
        Cached(filter=StandardScalerPlugin(), cache_data=True, cache_filter=True, overwrite=False, storage=None),
        ClassifierSVMPlugin(C=1.0, kernel='linear', gamma='scale')
    ],
    metrics=[F1(average='weighted')],
    overwrite=False,
    store=False,
    log=False
)

## 5. Entrenamiento del modelo

In [4]:
# Entrenar el modelo
pipeline.fit(x_train, y_train)

____________________________________________________________________________________________________

Fitting pipeline...

****************************************************************************************************

* Cached({'filter': StandardScalerPlugin({}), 'cache_data': True, 'cache_filter': True, 'overwrite': False, 
'storage': None}):

- El filtro StandardScalerPlugin({}) con hash 4f0150e0e11419085ce0f08ab077b7e5891f817b No existe, se va a 
entrenar.

...

	 * Saving in local path: cache/StandardScalerPlugin/4f0150e0e11419085ce0f08ab077b7e5891f817b/model
	 * Saved !


- El dato XYData(_hash='f77f9d95466939988cdd6a13f0cb91260b94c99d', 
_path='StandardScalerPlugin/4f0150e0e11419085ce0f08ab077b7e5891f817b') No existe, se va a crear.

- El dato XYData(_hash='f77f9d95466939988cdd6a13f0cb91260b94c99d', 
_path='StandardScalerPlugin/4f0150e0e11419085ce0f08ab077b7e5891f817b') Se cachea.

	 * Saving in local path: cache/StandardScalerPlugin/4f0150e0e11419085ce0f08ab077b7e5891f817b/f77f9d95466939988cdd6a13f0cb91260b94c99d
	 * Saved !


* ClassifierSVMPlugin({'C': 1.0, 'kernel': 'linear', 'gamma': 'scale'}):

## 6. Predicción y evaluación

In [5]:
# Realizar predicciones
predictions = pipeline.predict(x_test)

# Evaluar el modelo
evaluation = pipeline.evaluate(x_test, y_test, predictions)
print("Resultados de la evaluación:", evaluation)

____________________________________________________________________________________________________

Predicting pipeline...

****************************************************************************************************

* Cached({'filter': StandardScalerPlugin({}), 'cache_data': True, 'cache_filter': True, 'overwrite': False, 
'storage': None})

- El dato XYData(_hash='0403ab1f29eb3d1d59f857aa03f7af153d7ff357', 
_path='StandardScalerPlugin/4f0150e0e11419085ce0f08ab077b7e5891f817b') No existe, se va a crear.

...

* ClassifierSVMPlugin({'C': 1.0, 'kernel': 'linear', 'gamma': 'scale'})

____________________________________________________________________________________________________

Evaluating pipeline...

____________________________________________________________________________________________________

Resultados de la evaluación:
{'F1': 0.9664109121909632}

## 7. Demostración del uso de Cache

In [6]:
# Ejecutar el pipeline nuevamente
print("Segunda ejecución (debería usar datos en caché):")
# Crear el pipeline con Cache
pipeline = F3Pipeline(
    filters=[
        Cached(
            filter=StandardScalerPlugin(),
            cache_data=True,
            cache_filter=True,
            overwrite=False,
        ),
        KnnFilter(),
    ],
    metrics=[F1()],
)
print(pipeline)
pipeline.fit(x_train, y_train)
predictions = pipeline.predict(x_test)
evaluation = pipeline.evaluate(x_test, y_test, predictions)
print("Resultados de la evaluación:", evaluation)

Segunda ejecución (debería usar datos en caché):

F3Pipeline(
    filters=[
        Cached(filter=StandardScalerPlugin(), cache_data=True, cache_filter=True, overwrite=False, storage=None),
        KnnFilter(
            n_neighbors=5,
            weights='uniform',
            algorithm='auto',
            leaf_size=30,
            p=2,
            metric='minkowski',
            metric_params=None,
            n_jobs=None
        )
    ],
    metrics=[F1(average='weighted')],
    overwrite=False,
    store=False,
    log=False
)

____________________________________________________________________________________________________

Fitting pipeline...

****************************************************************************************************

...

* KnnFilter({'n_neighbors': 5, 'weights': 'uniform', 'algorithm': 'auto', 'leaf_size': 30, 'p': 2, 'metric': 
'minkowski', 'metric_params': None, 'n_jobs': None})

____________________________________________________________________________________________________

Evaluating pipeline...

____________________________________________________________________________________________________

Resultados de la evaluación:
{'F1': 1.0}

En esta segunda ejecución, deberías notar que el StandardScalerPlugin usa los datos en caché, lo que puede resultar en un tiempo de ejecución más rápido.